# Summarizing Monthly King St. Pilot Data Interactively

In [10]:
# import modules
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas as pd
import pandas.io.sql as pandasql
import statistics
import datetime
import ipywidgets as widgets
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.widgets import GraphWidget
import plotly.figure_factory as ff
from IPython.html import widgets
from IPython.display import display, clear_output
import sys

In [11]:
# connect to database
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

## The Dataframe

The frame containing March street data will be made with a pandas dataframe. 

In [12]:
# grab king_pilot.dash_daily_dev data
sql1 = pg.SQL("SELECT * from king_pilot.dash_daily_dev where category = 'Pilot'")
sql2 = pg.SQL("SELECT * from king_pilot.dash_baseline")
pilots = pandasql.read_sql(sql1, con)
baselines = pandasql.read_sql(sql2, con)

In [13]:
pilots

,street,direction,dt,day_type,category,period,tt
0,Dundas,Eastbound,2017-11-13,Weekday,Pilot,AM Peak,9.684035
1,Dundas,Eastbound,2017-11-14,Weekday,Pilot,AM Peak,11.630857
2,Dundas,Eastbound,2017-11-15,Weekday,Pilot,AM Peak,10.339368
3,Dundas,Eastbound,2017-11-16,Weekday,Pilot,AM Peak,11.222931
4,Dundas,Eastbound,2017-11-17,Weekday,Pilot,AM Peak,10.206434
5,Dundas,Eastbound,2017-11-20,Weekday,Pilot,AM Peak,11.094448
6,Dundas,Eastbound,2017-11-21,Weekday,Pilot,AM Peak,11.069244
7,Dundas,Eastbound,2017-11-22,Weekday,Pilot,AM Peak,11.248839
8,Dundas,Eastbound,2017-11-23,Weekday,Pilot,AM Peak,10.488873
9,Dundas,Eastbound,2017-11-24,Weekday,Pilot,AM Peak,10.657588


### Grabbing Specified Data from the Dataframe
The following function produces the mean travel time. from the datarame according to input specifications. If the user specifies a street, direction, day type, period, and whether the data of interes is pilot or baseline, the mean travel time will appear. An example for Dundas Eastbound Weekday AM Peak follows. 

In [14]:
def grab(st, direc, daytype, p, c):
    l = []
    if c == 'Baseline':
        for i in range(len(baselines)):
            if baselines.day_type.values[i] == daytype and baselines.period.values[i] == p and baselines.street.values[i] == st and baselines.direction.values[i] == direc:
                l.append(baselines.tt.values[i])
    elif c == 'Pilot':
        for i in range(len(pilots)):
            if pilots.day_type.values[i] == daytype and pilots.period.values[i] == p and pilots.street.values[i] == st and pilots.direction.values[i] == direc and pilots.dt.values[i] >= datetime.date(2018, 3, 1) and pilots.dt.values[i] <= datetime.date(2018, 3, 31):
                l.append(pilots.tt.values[i])        
    return round(statistics.mean(l), 2)

grab('Dundas', 'Westbound', 'Weekday', 'AM Peak', 'Baseline')


9.4299999999999997

We create an array of titles for columns of our dataframe. A generator expression is used to turn the array into a list of tuples that will be put into our dataframe later. 

In [15]:
valuesarr = [['East-West Corridors', 'Dundas', 'Dundas', 'Queen', 'Queen', 'Richmond', 
              'Adelaide', 'Wellington', 'Front', 'Front', 'North-South Corridors',
              'Bathurst', 'Bathurst', 'Spadina', 'Spadina', 
              'University',  'University', 'Yonge', 'Yonge', 'Jarvis', 'Jarvis'],
             [' ', 'EB', 'WB', 'EB', 'WB', 'WB', 'EB',  
              'WB', 'EB', 'WB', '', 'NB', 'SB', 'NB', 'SB', 
              'NB', 'SB', 'NB', 'SB', 'NB', 'SB' ],
             [' ','Bathurst', 'Jarvis', 'Bathurst', 'Jarvis', 'Jarvis', 'Spadina',
             'Jarvis', 'Bathurst', 'Yonge', '', 'Front', 'Dundas', 'Front', 'Dundas',
             'Front', 'Dundas', 'Front', 'Dundas', 'Front', 'Dundas'],
             [' ','Jarvis', 'Bathurst', 'Jarvis', 'Bathurst', 'Bathurst', 'Jarvis', 'Blue Jays',
             'Jarvis', 'Bathurst', '', 'Dundas', 'Front', 'Dundas', 'Front', 'Dundas', 'Front', 
              'Dundas', 'Front', 'Dundas', 'Front']]
tuples = list(zip(*valuesarr))

tuples

[('East-West Corridors', ' ', ' ', ' '),
 ('Dundas', 'EB', 'Bathurst', 'Jarvis'),
 ('Dundas', 'WB', 'Jarvis', 'Bathurst'),
 ('Queen', 'EB', 'Bathurst', 'Jarvis'),
 ('Queen', 'WB', 'Jarvis', 'Bathurst'),
 ('Richmond', 'WB', 'Jarvis', 'Bathurst'),
 ('Adelaide', 'EB', 'Spadina', 'Jarvis'),
 ('Wellington', 'WB', 'Jarvis', 'Blue Jays'),
 ('Front', 'EB', 'Bathurst', 'Jarvis'),
 ('Front', 'WB', 'Yonge', 'Bathurst'),
 ('North-South Corridors', '', '', ''),
 ('Bathurst', 'NB', 'Front', 'Dundas'),
 ('Bathurst', 'SB', 'Dundas', 'Front'),
 ('Spadina', 'NB', 'Front', 'Dundas'),
 ('Spadina', 'SB', 'Dundas', 'Front'),
 ('University', 'NB', 'Front', 'Dundas'),
 ('University', 'SB', 'Dundas', 'Front'),
 ('Yonge', 'NB', 'Front', 'Dundas'),
 ('Yonge', 'SB', 'Dundas', 'Front'),
 ('Jarvis', 'NB', 'Front', 'Dundas'),
 ('Jarvis', 'SB', 'Dundas', 'Front')]

## Dropdown Widgets

The following are dropdown widgets with daytypes and periods. Since periods depend on daytypes, the options in the period dropdown will change depending on whether 'Weekday' or 'Weekend'.This is done using the `on_value_change(change)` function.  

A button is made that will generate the dataframe based on user input. Using the `observe` method, the button will track changes made in the dropdowns. 

In [16]:
dtype={'Weekday':['AM Peak','Midday','PM Peak', 'Evening'],'Weekend':['Morning','Afternoon','Evening']}

def returnperiod(period):
    return period

def select_period(daytype):
    periodw.options = dtype[daytype]
    

button = widgets.Button(description="Generate Summary")


typew = widgets.Dropdown(options=dtype.keys(), description = 'Day Type')
periodw = widgets.Dropdown(options=dtype[typew.value], description = 'Period')

i = widgets.interactive(select_period, daytype=typew)
j = widgets.interactive(returnperiod, period=periodw)


def on_value_change(change):
    typew.value = change['new']
    periodw.value = change['new']

button.observe(on_value_change, names='value')

out = widgets.Output()

## Putting it All Together

The `on_button_clicked(b)` is the function for the button that will generate our dataframe. Within this function, a dataframe called `df` is created through several list variables. Essentially, each street is assigned a variable whose values are the results of the `grab()` function. For the `day_type` and `period` input, the dropdown values are selected. 

The column names for the data classifiers are made via the MultiIndex method built in pandas.

A **critical** part of this function is the `with out` portion. This portion of code allows one dataframe to be displayed, with the values within it changing when the button is clicked. 

*Without this portion of code, a new dataframe will appear every single time the button is clicked!*

In [17]:
def on_button_clicked(b):

    index = pd.MultiIndex.from_tuples(tuples, names=['Street Name', 'Direction', 'From', 'To'])
    
    #street variables containing mean travel times dependent on user selection of dropdown options 
    dundas = [grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Dundas', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    dundas.append(dundas[0]-dundas[1])
    dundas = dundas + [grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Dundas', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    dundas.append(dundas[3]-dundas[4])
    queen = [grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Queen', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    queen.append(queen[0]-queen[1])
    queen = queen + [grab('Queen', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Queen', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    queen.append(queen[3]-queen[4])
    richmond = [grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Richmond', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    richmond.append(richmond[0]-richmond[1])
    adelaide = [grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Adelaide', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    adelaide.append(adelaide[0]-adelaide[1])
    wellington = [grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Wellington', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    wellington.append(wellington[0]-wellington[1])
    front = [grab('Front', 'Eastbound',  typew.value , periodw.value, 'Pilot'), 
              grab('Front', 'Eastbound',  typew.value , periodw.value, 'Baseline')]
    front.append(front[0]-front[1])
    front = front + [grab('Front', 'Westbound',  typew.value , periodw.value, 'Pilot'),
              grab('Front', 'Westbound',  typew.value , periodw.value, 'Baseline')]
    front.append(front[3]-front[4])
    
    bathurst = [grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Pilot'),
           grab('Bathurst', 'Northbound', typew.value, periodw.value, 'Baseline')]
    bathurst.append(bathurst[0]-bathurst[1])
    bathurst = bathurst + [grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Pilot'),
                       grab('Bathurst', 'Southbound', typew.value, periodw.value, 'Baseline')]
    bathurst.append(bathurst[3]-bathurst[4])


    spadina = [grab('Spadina', 'Northbound', typew.value, periodw.value, 'Pilot'), 
              grab('Spadina', 'Northbound', typew.value, periodw.value, 'Baseline')]
    spadina.append(spadina[0]-spadina[1])
    spadina = spadina + [grab('Spadina', 'Southbound', typew.value, periodw.value, 'Pilot'),
                        grab('Spadina', 'Southbound', typew.value, periodw.value, 'Baseline')]
    spadina.append(spadina[3]-spadina[4])


    university = [grab('University', 'Northbound', typew.value, periodw.value, 'Pilot'), 
                 grab('University', 'Northbound', typew.value, periodw.value, 'Baseline')]
    university.append(university[0]-university[1])
    university = university + [grab('University', 'Southbound', typew.value, periodw.value, 'Pilot'),
                               grab('University', 'Southbound', typew.value, periodw.value, 'Baseline')]
    university.append(university[3]-university[4])


    yonge = [grab('Yonge', 'Northbound', typew.value, periodw.value, 'Pilot'),
            grab('Yonge', 'Northbound', typew.value, periodw.value, 'Baseline')]
    yonge.append(yonge[0]-yonge[1])
    yonge = yonge + [grab('Yonge', 'Southbound', typew.value, periodw.value, 'Pilot'),
                    grab('Yonge', 'Southbound', typew.value, periodw.value, 'Baseline')]
    yonge.append(yonge[3]-yonge[4])


    jarvis = [grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Pilot'),
             grab('Jarvis', 'Northbound', typew.value, periodw.value, 'Baseline')]
    jarvis.append(jarvis[0]-jarvis[1])
    jarvis = jarvis + [grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Pilot'),
             grab('Jarvis', 'Southbound', typew.value, periodw.value, 'Baseline')]
    jarvis.append(jarvis[3]-jarvis[4])
    
    # the data for the dataframe
    
    x = [[' ',' ',' ']]+[dundas[0:3],dundas[3:6],
                     queen[0:3], queen[3:6],
                     richmond[0:3],
                     adelaide[0:3], wellington[0:3],
                     front[0:3], front[3:6]] + [['','','']] + [bathurst[0:3], bathurst[3:6],
                     spadina[0:3], spadina[3:6],
                     university[0:3], university[3:6], 
                     yonge[0:3], yonge[3:6], 
                     jarvis[0:3], jarvis[3:6]]
    
    df = pd.DataFrame(x, index=index,
                      columns=['Pilot', 'Baseline', 'Change'])
    
    # delete existing data frame and create a new one with new information- i.e update dataframe
    
    with out:
        clear_output(True)
        display(df)



## The Interactive Summary

Now we just display the widgets. 

In [18]:
button.on_click(on_button_clicked)

display(i)
display(j)
display(button)
  
out

interactive(children=(Dropdown(description='Day Type', options=('Weekday', 'Weekend'), value='Weekday'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Dropdown(description='Period', options=('AM Peak', 'Midday', 'PM Peak', 'Evening'), value='AM Peak'), Output()), _dom_classes=('widget-interact',))

Button(description='Generate Summary', style=ButtonStyle())

Output()